# Missing Data and Imputation

Work in progress - converting the R script for missing data exploration and imputation into this jenky ass Colab script that makes everything a hundred times harder. Prioritizing the imputation right now so we can run it from Colab without shenanigans. Will add the graphs and tests here later.

Note that the FNN package was not in the tar.gz file I added to drive, so we still have to install it manually and it takes a couple of minutes. Will redo this and load it locally to speed things up eventually.

In [1]:
%load_ext rpy2.ipython

In [ ]:
%%R
# Need to get hard copy of FNN and dependencies into drive.
# For now installing, and it takes a long time
install.packages('FNN', dependencies = TRUE, repos = "https://cloud.r-project.org")

In [ ]:
# If in colab, mount drive and set wd
# If local, set wd to parent of notebook
import os
import re

try:
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/MyDrive/ds1_nhanes/')
except:
  from pathlib import Path
  if not re.search(r'ds1_nhanes$', str(os.getcwd())):
    os.chdir(Path(os.getcwd()).parent)

print(os.getcwd())

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
.libPaths(c("dev/r_library/library", .libPaths()))
print(.libPaths())

In [ ]:
%%R
library(dplyr)
library(missRanger)
library(stringr)

In [ ]:
%%R
raw <- read.csv('data/clean/nhanes_2017_2023_clean.csv')
str(raw)

In [ ]:
%%R
# Reduce to just the variables we want to impute - biomarkers and demographics
dat <- raw %>%
  select(
    gender:income_ratio,
    total_cholesterol,
    blood_mercury,
    avg_systolic_bp,
    avg_diastolic_bp
  )
str(dat)

In [ ]:
%%R
# missRanger imputation
imp <- missRanger(
  dat,
  pmm.k = 5,
  num.trees = 100,
  seed = 42,
  keep_forests = TRUE,
  verbose = 1
)
print(summary(imp))

# Check errors
print(imp$mean_pred_errors)

# Best errors
print(imp$pred_errors[2, ])

Put the DF back together so we can save it

In [ ]:
%%R

# Combine back with full
df <- raw %>%
  select(-all_of(c(names(imp$data))))
# str(df)

# Combine
df <- bind_cols(df, imp$data)
# str(df)

# Remove extra biomarkers
# Also removing income ratio qs - will have to recreate it after imp
df <- df %>%
  select(
      -c(cholesterol_std_dev:serum_ferritin),
      -income_ratio_qs
    )
# str(df)

# Get parenthesis back into col names
# (R automatically converted them to periods)
names(df) <- names(df) %>%
  str_replace('\\.', '\\(') %>%
  str_replace('eq\\.', 'eq\\)') %>%
  str_replace('drinks\\.', 'drinks\\)') %>%
  str_replace('grams\\.', 'grams\\)')
str(df)

Save as csv

In [ ]:
%%R
write.csv(df, 'data/clean/nhanes_2017_2023_imputed.csv')